In [38]:
def parse_fix_message(message):
    fields = message.split('|')
    fields_by_tag = {}
    group_counts = {}

    for field in fields:
      if "=" in field:
        tag, value = field.split('=')
        if tag in fields_by_tag:
            return f'Error: tag {tag} repeated in message {message}'
        if tag.isdigit() and int(tag) in [279, 280]:
            count = value.count('\x01') + 1
            if tag in group_counts and group_counts[tag] != count:
              return f'ERROR: Repeating group with different count in message {message}'
              break
            else:
              group_counts[tag] = count
        fields_by_tag[tag] = value
    return fields_by_tag


def process_fix_messages(filepath):
    # Initialize variables to store results
    error_messages = []
    prices_by_account = {}

    with open(filepath, 'r') as f:
        for line in f:
            if not line:
                continue
            if line[-1] == '\n':
                line = line[:-1]

            # Parse the message
            fields = parse_fix_message(line)
            if isinstance(fields, str):
                error_messages.append(fields)
                continue

            # Check if the message is a New Order Single
            if fields['35'] == 'D':
                account = fields['1']
                price = float(fields['44'])
                if account not in prices_by_account:
                    prices_by_account[account] = {'highest': price, 'lowest': price}
                else:
                    prices_by_account[account]['highest'] = max(prices_by_account[account]['highest'], price)
                    prices_by_account[account]['lowest'] = min(prices_by_account[account]['lowest'], price)

    return error_messages, prices_by_account


if __name__ == '__main__':
    filepath = 'Fix.Sample.txt'
    error_messages, prices_by_account = process_fix_messages(filepath)

    # Output error messages
    for error in error_messages:
        print(error)
        print()

    # Output highest and lowest prices by account
    for account in prices_by_account:
        print(f'Account {account}: highest price={prices_by_account[account]["highest"]}, lowest price={prices_by_account[account]["lowest"]}')


Error: tag 600 repeated in message 8=FIX.4.2|9=0314|35=AB|34=000000011|1=SPDRT1|11=10109A9578A82AAA|100=1|21=1|38=10|40=2|44=-3.98|54=1|55=BA|59=0|167=MLEG|207=XASE|60=20181127-16:46:07.996|204=8|555=2|600=BA|608=OP|609=OPT|611=20181109|612=360|654=EA628EA5B0F8186|623=1|624=2|564=O|600=BA|608=OP|609=OPT|611=20181109|612=362.5|654=EA628EA5B0F8187|623=1|624=2|564=O|10=195|

Error: tag 600 repeated in message 8=FIX.4.2|9=0308|35=AB|34=000000000|1=H20466|11=AFCDAF7E0FC8141|100=N|21=1|38=50|40=2|44=-11.7|54=1|55=NVDA|59=0|167=MLEG|60=20181127-16:46:07.996|204=0|555=3|600=NVDA|608=OP|609=OPT|611=20200117|612=215|654=EA61EEBE5226038|623=2|624=1|564=O|600=NVDA|608=OP|609=OPT|611=20210115|612=190|654=EA61EEBE5226039|623=1|624=2|564=O|10=078|

Error: tag 9303 repeated in message 8=FIX.4.2|9=0151|35=D|34=000000014|1=SPDRT1|11=DB95A6B1070A29C|9303=Y|38=100|40=2|44=266.04|54=1|55=SPY|59=0|60=20181127-16:46:07.996|100=XNYS|9416=A|9303=N|528=A|386=1|336=2|10=028|

Error: tag 55 repeated in message 8=